In [ ]:
import pandas as pd
import numpy as np
import bayesian_changepoint_detection.offline_changepoint_detection as offcd
import bayesian_changepoint_detection.hazard as hazard
import bayesian_changepoint_detection.distributions as distributions
from functools import partial
import ruptures as rpt

def evaluate_change_points(est_CP, true_CP, T):
    tau = T - 1
    num_CP = len(est_CP)

    if num_CP == 0:
        dist_est_gt = np.inf
        dist_gt_est = -np.inf
        covering_metric = 0
    else:
        # max over true CPs of distance to nearest est CP
        holder_est_gt = [min(abs(np.array(est_CP) - i)) for i in true_CP]
        dist_est_gt = max(holder_est_gt) if holder_est_gt else np.inf

        # max over est CPs of distance to nearest true CP
        holder_gt_est = [min(abs(np.array(true_CP) - i)) for i in est_CP]
        dist_gt_est = max(holder_gt_est) if holder_gt_est else np.inf

        # add boundaries
        gt_CP_all = [1] + list(true_CP) + [T + 1]
        est_CP_all = [1] + list(est_CP) + [T + 1]

        # construct segment index sets
        gt_list = [set(range(gt_CP_all[i-1], gt_CP_all[i])) for i in range(1, len(gt_CP_all))]
        est_list = [set(range(est_CP_all[i-1], est_CP_all[i])) for i in range(1, len(est_CP_all))]

        covering_metric = 0
        for A in gt_list:
            jaccard_vals = []
            for Ap in est_list:
                inter = len(A & Ap)
                union = len(A | Ap)
                jaccard_vals.append(inter / union if union > 0 else 0)
            covering_metric += len(A) * max(jaccard_vals)
        covering_metric /= (tau + 1)

    abs_error = abs(num_CP - len(true_CP))

    return pd.DataFrame([{
        "abs_error": abs_error,
        "dist_est_gt": dist_est_gt,
        "dist_gt_est": dist_gt_est,
        "covering_metric": covering_metric,
        "est_CP": est_CP
    }])



Y_flat = pd.read_csv("./reps_sim_dat/y_rep0.csv").values  # shape (20000, 3)


T, N, d = 200, 100, 3
Y_array = Y_flat.reshape(T, N, d)
Y_mean = Y_array.mean(axis=1) 

# Method1: Ruptures
# Truong, C., Oudre, L., & Vayatis, N. (2018). ruptures: change point detection in Python. arXiv preprint arXiv:1801.00826.
algo = rpt.Binseg(model="l2").fit(Y_mean)
bkps = algo.predict(n_bkps=3)
est_CP = [b + 1 for b in bkps[:-1]]
res_ruptures = evaluate_change_points(est_CP, [51,101,151], 200)
res_ruptures["method"] = "ruptures"
res_ruptures

# Method2: bayesian-changepoint-detection   #########Only for Univariate Case###############
# Adams, R. P., & MacKay, D. J. C. (2007).Bayesian Online Changepoint Detection. arXiv preprint arXiv:0710.3742.




,abs_error,dist_est_gt,dist_gt_est,covering_metric,est_CP,method
0,0,0,0,1.0,"[51, 101, 151]",ruptures


In [10]:
bkps[0:3]

[50, 100, 150]